dict keys (can be used in fetch_stock_metrics to extract useful signals)
dict_keys(['zip', 'sector', 'fullTimeEmployees', 'longBusinessSummary', 'city', 'phone', 'state', 'country', 'companyOfficers', 'website', 'maxAge', 'address1', 'industry', 'address2', 'ebitdaMargins', 'profitMargins', 'grossMargins', 'operatingCashflow', 'revenueGrowth', 'operatingMargins', 'ebitda', 'targetLowPrice', 'recommendationKey', 'grossProfits', 'freeCashflow', 'targetMedianPrice', 'currentPrice', 'earningsGrowth', 'currentRatio', 'returnOnAssets', 'numberOfAnalystOpinions', 'targetMeanPrice', 'debtToEquity', 'returnOnEquity', 'targetHighPrice', 'totalCash', 'totalDebt', 'totalRevenue', 'totalCashPerShare', 'financialCurrency', 'revenuePerShare', 'quickRatio', 'recommendationMean', 'exchange', 'shortName', 'longName', 'exchangeTimezoneName', 'exchangeTimezoneShortName', 'isEsgPopulated', 'gmtOffSetMilliseconds', 'quoteType', 'symbol', 'messageBoardId', 'market', 'annualHoldingsTurnover', 'enterpriseToRevenue', 'beta3Year', 'enterpriseToEbitda', '52WeekChange', 'morningStarRiskRating', 'forwardEps', 'revenueQuarterlyGrowth', 'sharesOutstanding', 'fundInceptionDate', 'annualReportExpenseRatio', 'totalAssets', 'bookValue', 'sharesShort', 'sharesPercentSharesOut', 'fundFamily', 'lastFiscalYearEnd', 'heldPercentInstitutions', 'netIncomeToCommon', 'trailingEps', 'lastDividendValue', 'SandP52WeekChange', 'priceToBook', 'heldPercentInsiders', 'nextFiscalYearEnd', 'yield', 'mostRecentQuarter', 'shortRatio', 'sharesShortPreviousMonthDate', 'floatShares', 'beta', 'enterpriseValue', 'priceHint', 'threeYearAverageReturn', 'lastSplitDate', 'lastSplitFactor', 'legalType', 'lastDividendDate', 'morningStarOverallRating', 'earningsQuarterlyGrowth', 'priceToSalesTrailing12Months', 'dateShortInterest', 'pegRatio', 'ytdReturn', 'forwardPE', 'lastCapGain', 'shortPercentOfFloat', 'sharesShortPriorMonth', 'impliedSharesOutstanding', 'category', 'fiveYearAverageReturn', 'previousClose', 'regularMarketOpen', 'twoHundredDayAverage', 'trailingAnnualDividendYield', 'payoutRatio', 'volume24Hr', 'regularMarketDayHigh', 'navPrice', 'averageDailyVolume10Day', 'regularMarketPreviousClose', 'fiftyDayAverage', 'trailingAnnualDividendRate', 'open', 'toCurrency', 'averageVolume10days', 'expireDate', 'algorithm', 'dividendRate', 'exDividendDate', 'circulatingSupply', 'startDate', 'regularMarketDayLow', 'currency', 'regularMarketVolume', 'lastMarket', 'maxSupply', 'openInterest', 'marketCap', 'volumeAllCurrencies', 'strikePrice', 'averageVolume', 'dayLow', 'ask', 'askSize', 'volume', 'fiftyTwoWeekHigh', 'fromCurrency', 'fiveYearAvgDividendYield', 'fiftyTwoWeekLow', 'bid', 'tradeable', 'dividendYield', 'bidSize', 'dayHigh', 'regularMarketPrice', 'logo_url'])

In [1]:
import yfinance as yf
from dotenv import load_dotenv
import pandas as pd
import time
import os
import boto3
from io import StringIO  
import json
from datetime import datetime, timedelta

# Load environment variables from the .env file
load_dotenv()

# Access the environment variables for AWS credentials
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_DEFAULT_REGION = os.getenv('AWS_DEFAULT_REGION')

start_date=(datetime.today() - timedelta(days=10*365)).strftime('%Y-%m-%d')
end_date=datetime.today().strftime('%Y-%m-%d')
# Create an S3 client using the loaded credentials
s3_client = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_DEFAULT_REGION
)

# AWS S3 Bucket Name
s3_bucket_name = 'isbfinanceproject'  # Replace with your S3 bucket name

s3_client.put_bucket_versioning(
    Bucket=s3_bucket_name,
    VersioningConfiguration={
        'Status': 'Enabled'
    }
)

# Define the tickers for each vertical
tickers = {
    "Energy": ["RELIANCE.NS", "NTPC.NS", "ONGC.NS", "IOCL.NS", "BPCL.NS", "GAIL.NS", "TATAPOWER.NS", "ADANIGREEN.NS", "MGL.NS", "HINDPETRO.NS",
               "APOLLOPIPE.NS", "BIRLACORPN.NS", "CIL.NS", "HINDALCO.NS", "SAIL.NS", "JSWSTEEL.NS", "TATAMOTORS.NS", "VEDANTA.NS", "OIL.NS", "NHPC.NS"],
    "Pharmacy": ["SUNPHARMA.NS", "CIPLA.NS", "DIVISLAB.NS", "LUPIN.NS", "DRREDDY.NS", "WOKHARDT.NS", "ABBOTINDIA.NS", "PIRAMAL.NS", "MOTHERSONSumi.NS", "ZYDUSLIFE.NS",
                 "ALKEM.NS", "BIOCON.NS", "NATCOPHARMA.NS", "FDC.NS", "SYNGENE.NS", "MANKIND.NS", "GLENMARK.NS", "CADILAHC.NS", "TORNTPHARM.NS", "KOTAKPHARMA.NS"],
    "Automobile": ["MARUTI.NS", "TATAMOTORS.NS", "MAHINDRA.NS", "BAJAJ-AUTO.NS", "EICHERMOT.NS", "MOTHERSUMI.NS", "TVSMOTOR.NS", "ASHOKLEY.NS", "AUBANK.NS", "BOSCHLTD.NS",
                   "EXIDEIND.NS", "INDIGO.NS", "OLECTRA.NS", "HERO.MOTOCO.NS", "RECLTD.NS", "GMDCLTD.NS", "SONALIKA.NS", "TATAPOWER.NS", "BHEL.NS", "HAVELLS.NS"],
    "Banking": ["HDFCBANK.NS", "ICICIBANK.NS", "AXISBANK.NS", "SBIN.NS", "KOTAKBANK.NS", "IDFCFIRSTB.NS", "INDUSINDBK.NS", "YESBANK.NS", "BANKBARODA.NS", "PNB.NS",
                "BOMDIA.NS", "RBLBANK.NS", "FEDERALBNK.NS", "SOUTHBANK.NS", "CANBK.NS", "IIB.NS", "MUTHOOTFIN.NS", "LICHSGFIN.NS", "NHB.NS", "BANDHANBNK.NS"],
    "Technology": ["TCS.NS", "INFY.NS", "WIPRO.NS", "HCLTECH.NS", "TECHM.NS", "MINDTREE.NS", "LTTS.NS", "COFORGE.NS", "CYIENT.NS", "NIIT.NS",
              "MATRIXLABS.NS", "BIRLASOFT.NS", "KPITTECH.NS", "ZENSARTECH.NS", "OCTALIT.NS", "TATAELXSI.NS", "SASKEN.NS", "3IINFOTECH.NS", "INTELLECT.NS"]
}

# Function to download stock data (historical price data)
def fetch_stock_data(ticker, start_date=start_date, end_date=end_date):
    try:
        data = yf.download(ticker, start=start_date, end=end_date)
        return data
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

# Function to fetch financial metrics like P/E ratio, Beta, Market Cap, EPS, etc.
def fetch_stock_metrics(ticker):
    try:
        stock_info = yf.Ticker(ticker)
        
        # Extracting key financial metrics
        metrics = {
            "P/E Ratio": stock_info.info.get('trailingPE', 'N/A'),
            "Beta (5Y Monthly)": stock_info.info.get('beta', 'N/A'),
            "Market Cap": stock_info.info.get('marketCap', 'N/A'),
            "Forward P/E": stock_info.info.get('forwardPE', 'N/A'),
            "EPS (TTM)": stock_info.info.get('trailingEps', 'N/A'),
            "Price": stock_info.info.get('previousClose', 'N/A'),
            "Sector": stock_info.info.get('sector', 'N/A'),
            "Industry": stock_info.info.get('industry', 'N/A')
        }
        
        return metrics
    except Exception as e:
        print(f"Error fetching metrics for {ticker}: {e}")
        return None


def upload_to_s3(data, bucket_name, file_name, file_format='csv'):
    if file_format == 'csv':

        csv_buffer = StringIO()
        data.to_csv(csv_buffer)
        s3_client.put_object(Bucket=bucket_name, Key=file_name, Body=csv_buffer.getvalue())
        print(f"Data uploaded to S3: {file_name} (CSV format)")

    elif file_format == 'json':
        json_data = json.dumps(data, indent=4)
        s3_client.put_object(Bucket=bucket_name, Key=file_name, Body=json_data)
        print(f"Data uploaded to S3: {file_name} (JSON format)")

def process_tickers(tickers, start_date=start_date, end_date=end_date):
    for vertical, ticker_list in tickers.items():
        for ticker in ticker_list:
            print(f"Fetching data for {ticker}...")
            
            # Fetch stock data (price history)
            data = fetch_stock_data(ticker, start_date, end_date)
            
            
            metrics = fetch_stock_metrics(ticker)
            
            if data is not None and metrics is not None:
            #     # Add metrics as additional columns to the stock data
            #     for key, value in metrics.items():
            #         data[key] = value  
                
                # Construct the S3 file path for historical stock data (CSV format)
                s3_file_name = f"{vertical}/{ticker}/{ticker}_data.csv"
                upload_to_s3(data, s3_bucket_name, s3_file_name, file_format='csv')
                
                # Construct the S3 file path for stock metrics (JSON format)
                metrics_file_name = f"{vertical}/{ticker}/{ticker}_metrics.json"
                upload_to_s3(metrics, s3_bucket_name, metrics_file_name, file_format='json')
            
            # Implementing the delay for request rate limiting
            time.sleep(1)  # Adjust time if needed based on your request limit

# Run the pipeline and upload stock data and metrics for all companies to S3
process_tickers(tickers)

print("Stock data fetching and upload to S3 complete.")


Fetching data for RELIANCE.NS...


[*********************100%***********************]  1 of 1 completed


Data uploaded to S3: Energy/RELIANCE.NS/RELIANCE.NS_data.csv (CSV format)
Data uploaded to S3: Energy/RELIANCE.NS/RELIANCE.NS_metrics.json (JSON format)
Fetching data for NTPC.NS...


[*********************100%***********************]  1 of 1 completed


Data uploaded to S3: Energy/NTPC.NS/NTPC.NS_data.csv (CSV format)
Data uploaded to S3: Energy/NTPC.NS/NTPC.NS_metrics.json (JSON format)
Fetching data for ONGC.NS...


...

Data uploaded to S3: Technology/SASKEN.NS/SASKEN.NS_data.csv (CSV format)
Data uploaded to S3: Technology/SASKEN.NS/SASKEN.NS_metrics.json (JSON format)
Fetching data for 3IINFOTECH.NS...


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['3IINFOTECH.NS']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


Data uploaded to S3: Technology/3IINFOTECH.NS/3IINFOTECH.NS_data.csv (CSV format)
Data uploaded to S3: Technology/3IINFOTECH.NS/3IINFOTECH.NS_metrics.json (JSON format)
Fetching data for INTELLECT.NS...


[*********************100%***********************]  1 of 1 completed


Data uploaded to S3: Technology/INTELLECT.NS/INTELLECT.NS_data.csv (CSV format)
Data uploaded to S3: Technology/INTELLECT.NS/INTELLECT.NS_metrics.json (JSON format)
Stock data fetching and upload to S3 complete.
